In [6]:
import torch
tokenizer = torch.hub.load('huggingface/pytorch-pretrained-BERT', 'bertTokenizer', 'bert-base-cased', do_basic_tokenize=False)

# Tokenized input
text = "[CLS] Who was Jim Henson ? [SEP] Jim Henson was a puppeteer [SEP]"
tokenized_text = tokenizer.tokenize(text)
indexed_tokens = tokenizer.convert_tokens_to_ids(tokenized_text)

Using cache found in /Users/mohamedkhodeir/.cache/torch/hub/huggingface_pytorch-pretrained-BERT_master
The pre-trained model you are loading is a cased model but you have not set `do_lower_case` to False. We are setting `do_lower_case=False` for you but you may want to check this behavior.
100%|██████████| 213450/213450 [00:03<00:00, 58028.97B/s]


In [9]:
tokenized_text

['[CLS]',
 'Who',
 'was',
 'Jim',
 'He',
 '##nson',
 '?',
 '[SEP]',
 'Jim',
 'He',
 '##nson',
 'was',
 'a',
 'puppet',
 '##eer',
 '[SEP]']

In [7]:
### Get the hidden states computed by `bertModel`
# Define sentence A and B indices associated to 1st and 2nd sentences (see paper)
segments_ids = [0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1]

# Convert inputs to PyTorch tensors
segments_tensors = torch.tensor([segments_ids])
tokens_tensor = torch.tensor([indexed_tokens])

model = torch.hub.load('huggingface/pytorch-pretrained-BERT', 'bertModel', 'bert-base-cased')
model.eval()

with torch.no_grad():
    encoded_layers, _ = model(tokens_tensor, segments_tensors)

Using cache found in /Users/mohamedkhodeir/.cache/torch/hub/huggingface_pytorch-pretrained-BERT_master
100%|██████████| 435779157/435779157 [02:06<00:00, 3458177.72B/s]


In [8]:
encoded_layers

tensor([[[ 0.0671, -0.0819, -0.3849,  ...,  0.2479,  0.8418,  0.5378],
         [-0.0828, -0.2384,  0.3622,  ..., -0.0386, -0.5637,  0.1459],
         [ 0.0357, -0.1931,  0.3957,  ...,  0.4133, -0.1263, -0.0303],
         ...,
         [ 0.1778,  0.0921, -0.0885,  ...,  0.6259, -0.3713,  0.0748],
         [ 0.0062, -0.4483, -0.4176,  ...,  0.1641, -0.1112, -0.0908],
         [ 0.3182, -0.0281, -0.4652,  ...,  0.5634,  1.3185,  0.6513]]])

In [1]:
import torch


In [3]:
question = torch.randn((10, 32, 200))
document = torch.randn((500, 32, 200))

In [37]:
context_matrix = torch.randn(200, 200)

In [38]:
doc_con = torch.matmul(document, context_matrix)

In [40]:
doc_con[0,0] == context_matrix.mv(document[0,0])

tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0], dtype=torch.uint8)

In [50]:
context_matrix.unsqueeze(0).shape

torch.Size([1, 200, 200])

In [51]:
doc_con = torch.matmul(context_matrix, document.view(-1, 200))

RuntimeError: Expected tensor to have size 200 at dimension 1, but got size 16000 for argument #2 'batch2' (while checking arguments for bmm)

In [73]:
doc_con = torch.einsum('qh,mnh->mnq',context_matrix, document)

In [76]:
(context_matrix.mv(document[0,3]) - doc_con[0,3]).max()

tensor(1.1444e-05)

In [85]:
question[0].shape, doc_con.shape

(torch.Size([32, 200]), torch.Size([500, 32, 200]))

In [91]:
a = torch.einsum('nh,mnh->mn',question[0], doc_con)

In [94]:
a.shape

torch.Size([500, 32])

In [96]:
question.shape

torch.Size([10, 32, 200])

In [101]:
question.transpose(0, 1).reshape(32, 2000).shape

torch.Size([32, 2000])

In [93]:
a[0,1] - question[0,1].dot(doc_con[0,1])

tensor(6.1035e-05)

In [79]:
question.transpose(0, 1).shape, doc_con.permute(1, 2, 0).shape
#.bmm(doc_con.permute(1, 2, 0))

(torch.Size([32, 10, 200]), torch.Size([32, 200, 500]))

In [64]:
question.transpose(0, 1).bmm(doc_con.permute(1, 2, 0))

torch.Size([200])

In [2]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

class TaskSpecificAttention(nn.Module):
	def __init__(self, input_size, projection_size):
		super(TaskSpecificAttention, self).__init__()
		self.input_size = input_size
		self.projection_size = projection_size
		self.context_vector = torch.randn((1, 1, projection_size), requires_grad=True)
		self.input_projection = nn.Tanh(nn.Linear(input_size, projection_size))
		self.softmax = nn.Softmax()

	def forward(self, input_seq):
		'''inputs should be [seq_length, batch_size, input_size]'''
		vector_attention = self.input_projection(input_seq) # should be [seq_length, batch_size, output_size]
		attention_weights = self.softmax((vector_attention * self.context_vector).sum(2, keepdim=True), dim=0) # should be [seq_length, batch_size, 1]
		return attention_weights

class BiLinearAttention(nn.Module):
	def __init__(self, input_size):
		super(BiLinearAttention, self).__init__()
		self.input_size = input_size

		self.context_matrix = torch.randn((input_size, input_size), requires_grad=True)
		self.softmax = nn.Softmax(dim=1)

	def forward(self, question, document):
		'''inputs should be [seq_length, batch_size, input_size]'''
		document_context = torch.matmul(document, self.context_matrix)
		attention_matrix = question.transpose(0, 1).bmm(document_context.permute(1, 2, 0))
		attention_weights = self.softmax(attention_matrix.sum(1, keepdim=True).permute(2,0,1))
		return attention_weights


class AttentiveReader(nn.Module):
    def __init__(self, embedding_dim, hidden_dim, vocab_size, lstm_layers=1, lstm_bidirectional=True):
        super(AttentiveReader, self).__init__()
        self.hidden_dim = hidden_dim

        self.word_embeddings = nn.Embedding(vocab_size, embedding_dim)

        self.question_lstm = nn.LSTM(input_size=embedding_dim, hidden_size=hidden_dim, num_layers=lstm_layers, bidirectional=lstm_bidirectional)
        self.document_lstm = nn.LSTM(input_size=embedding_dim, hidden_size=hidden_dim, num_layers=lstm_layers, bidirectional=lstm_bidirectional)
        self.attention = BiLinearAttention(input_size=hidden_dim * 2 if lstm_bidirectional else 1)

    def forward(self, question, document):
        question_embedding = self.word_embeddings(question)
        document_embedding = self.word_embeddings(document)
        print(question_embedding.shape)

        question_encoding, _ = self.question_lstm(question_embedding)
        document_encoding, _ = self.document_lstm(document_embedding)

        attention = self.attention(question_encoding, document_encoding)
        print(attention.shape, document_encoding.shape)
        output = (attention * document_encoding).sum(2)

        return output




In [3]:
x = AttentiveReader(100, 100, 50000)

In [4]:
question = torch.randint(low=0, high=50000, size=((10,32)))
document = torch.randint(low=0, high=50000, size=((200,32)))

In [6]:
x(question, document).shape

torch.Size([10, 32, 100])
torch.Size([200, 32, 1]) torch.Size([200, 32, 200])


torch.Size([200, 32])

In [7]:
!pip install nltk

    100% |████████████████████████████████| 1.5MB 170kB/s ta 0:00:01
  Running setup.py bdist_wheel for nltk ... done
  Stored in directory: /Users/mohamedkhodeir/Library/Caches/pip/wheels/41/c8/31/48ace4468e236e0e8435f30d33e43df48594e4d53e367cf061
Successfully built nltk
You are using pip version 10.0.1, however version 19.1.1 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


In [9]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to
[nltk_data]     /Users/mohamedkhodeir/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [10]:
from nltk.tokenize import sent_tokenize, word_tokenize

data = "All work and no play makes jack a dull boy, all work and no play"
print(word_tokenize(data))


['All', 'work', 'and', 'no', 'play', 'makes', 'jack', 'a', 'dull', 'boy', ',', 'all', 'work', 'and', 'no', 'play']


In [11]:
import pickle

In [12]:
pickle.dump?

In [13]:
import numpy as np
def loadGloveModel(gloveFile):
    print("Loading Glove Model")
    f = open(gloveFile,'r')
    model = {}
    for line in f:
        splitLine = line.split()
        word = splitLine[0]
        embedding = np.array([float(val) for val in splitLine[1:]])
        model[word] = embedding
    print("Done.",len(model)," words loaded!")
    return model

In [14]:
model = loadGloveModel('data/glove.6B.100d.txt')

Loading Glove Model
Done. 400000  words loaded!


In [15]:
embed = nn.Embedding(50000, 100)

In [17]:
embed.weight.shape

torch.Size([50000, 100])

In [19]:
from sklearn.feature_extraction.text import TfidfVectorizer
corpus = [
     'This is the first document.',
     'This document is the second document.',
     'And this is the third one.',
     'Is this the first document?',
 ]
def corpus_iterator():
    for i in corpus:
        yield i

vectorizer = TfidfVectorizer()
X = vectorizer.fit_transform(corpus_iterator())
print(vectorizer.get_feature_names())


['and', 'document', 'first', 'is', 'one', 'second', 'the', 'third', 'this']


In [22]:
vec = pickle.load(open('data/count_vectorizer.pkl', 'rb'))

In [26]:
vec.get_feature_names()[-3]

'zubaydah'

In [3]:
from utils.build_vocab import *

In [4]:
# glove_embeddings = load_glove_model()
data_vocab = load_count_vectorizer_data_vocab()

In [84]:
count_vectorizer = data_vocab['vectorizer']
counts = data_vocab['counts']

In [56]:
vectorizer = TfidfVectorizer()
counts = vectorizer.fit_transform(data_iterator())

In [89]:
x = np.asarray(counts.sum(0)).flatten()/tfidf.idf_
tfidf_sorting = np.argsort(x)[::-1]
feature_array[tfidf_sorting[:limit]]

array(['the', 'of', 'to', 'and', 'in', 'that', 'for', 'is', 'on', 'it',
       'with', 'as', 'by', 'this', 'was', 'have', 'are', 'be', 'we',
       'not', 'or', 'from', 'at', 'you', 'he', 'an', 'but', 'they', 'has',
       'said', 'who', 'about', 'more', 'his', 'all', 'would', 'will',
       'their', 'were', 'one', 'which', 'people', 'what', 'been', 'there',
       'our', 'if', 'state', 'other', 'than', 'so', 'had', 'can', 'trump',
       'no', 'when', 'president', 'also', 'new', 'do', 'out', 'up',
       'year', 'states', 'any', 'time', 'some', 'after', 'over',
       'because', 'years', 'its', 'these', 'those', 'may', 're', 'she',
       'them', 'just', 'first', 'into', 'now', 'two', 'only', 'percent',
       'under', 'law', 'such', 'how', 'like', '000', 'most', 'united',
       'her', 'going', 'health', 'government', 'many', 'know', 'could',
       'tax', 'us', 'well', 'think', 'very', 'public', 'get', 'country',
       'make', 'did', 'federal', 'house', 'national', 'your', 'should'

In [55]:
from sklearn.feature_extraction.text import TfidfTransformer, TfidfVectorizer

In [7]:
tfidf = TfidfTransformer().fit(counts)

In [90]:
response = tfidf.transform(counts)

In [9]:
feature_array = np.array(count_vectorizer.get_feature_names())

In [11]:
len(feature_array)

365905

In [10]:
response.shape

(93235, 365905)

In [91]:
tfidf_sorting = np.argsort(np.asarray(response.sum(0))).flatten()[::-1]

In [92]:
limit = 500

In [93]:
feature_array[tfidf_sorting[:limit]]

array(['the', 'to', 'of', 'and', 'in', 'that', 'for', 'is', 'on', 'it',
       'he', 'was', 'as', 'with', 'trump', 'we', 'are', 'by', 'said',
       'have', 'you', 'this', 'be', 'from', 'not', 'at', 'or', 'his',
       'has', 'they', 'an', 'but', 'who', 'will', 'would', 'state',
       'about', 'our', 'more', 'president', 'tax', 'their', 'people',
       'were', 'she', 'all', 'percent', 'health', 'new', 'had', 'than',
       'clinton', 'one', 'been', 'states', 'her', 'obama', 'there', 'if',
       'what', 'year', 'can', 'which', 'no', 'when', 'law', 'so', 'up',
       'out', 'other', 'also', '000', 'do', 'bill', 'says', 'million',
       'after', 'years', 'its', 'house', 'your', 'government', 'over',
       'federal', 'united', 'time', 'any', 'some', 'care', 're',
       'because', 'us', 'these', 'just', 'first', 'those', 'them', 'news',
       'may', 'country', 'public', 'now', 'like', 'into', '2016',
       'national', 'american', 'two', 'senate', 'only', 'campaign',
       'school',

In [80]:
feature_array[tfidf_sorting[-limit:]]

array(['critiquait', 'tirage', 'déplacés', 'démocratiques', 'neutre',
       'augmen', 'démontre', 'chiffre', 'dirais', 'soyons', 'démontrent',
       'alarme', 'chiffré', 'chiffrées', 'souviens', 'délicatesse',
       'pâturer', 'raffinage', 'dégoûtés', 'diminue', 'diminuer', 'misé',
       'rouvert', 'rouvrir', 'qualitatif', 'pêcheries', 'amortit',
       'faillites', 'pérennes', 'pénaliser', 'délicat', 'faible',
       'délicate', 'fleuve', 'continuellement', 'renié', 'favorisions',
       'habiller', 'importerait', 'instabilité', 'relancer', 'concrète',
       'dépassé', 'rejette', 'habille', 'vrac', 'favoritisme', 'sinécure',
       'souffle', 'concrets', 'autochtone', 'catégorie', 'solide',
       'réformiste', 'déchirer', 'référendaires', 'passés', 'écologistes',
       'écosystèmes', 'écologique', 'désolant', 'diffuseurs',
       'réductions', 'autochtones', 'achats', 'amenée', 'fassent',
       'amené', '050217', 'vare', 'enfocrment', 'ecpd', 'fods',
       'addtional', 'sherr

In [83]:
len(tfidf.idf_)

365905

In [44]:
stop_n = feature_array[tfidf_sorting[:limit]]

In [45]:
len(set(top_n))

500

In [46]:
len(stop_n)

500

In [48]:
stop_n

array(['gibson', 'orwell', 'the', 'parkland', 'novel', 'maine', '1984',
       'of', 'and', 'skinhead', 'leslie', 'to', 'is', 'george', 'alyssa',
       'theatre', 'smartphones', 'he', 'candidate', 'astronaut',
       'students', 'lesbian', 'gonzalez', 'in', 'predicts', 'shooting',
       'greene', 'emma', 'survivor', 'sabattus', 'twitter', 'carson',
       'nasa', 'insulted', 'brother', 'said', 'dystopian', 'fogg',
       'named', 'newspeak', 'doublethink', 'by', 'that', 'winston',
       'trained', 'activist', 'girl', 'as', 'english', 'she', 'an', 'who',
       'gilchrist', 'line', 'icke', 'hogg', 'old', 'screens', 'it',
       'macmillan', '57th', 'liar', 'world', 'arcadia', 'telescreen',
       'big', 'sturridge', 'race', 'novels', 'from', 'not', 'legislature',
       'for', 'thoughtcrime', 'republican', 'become', 'his', '17', 'bald',
       'power', 'one', 'everyman', 'mcmahon', 'persecutes', 'being',
       'are', 'via', 'airstrip', 'was', 'but', 'surveillance', 'calling',
      

In [47]:
top_n == stop_n

array([False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False,

In [ ]:
top_n = feature_array[tfidf_sorting[:limit]]